In [9]:
import tensorflow as tf

from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np

from sklearn.preprocessing import StandardScaler

import pandas as pd

df = pd.read_csv("data/final_train.csv")

trY = df['Item_Outlet_Sales']

df.drop(['Item_Identifier'], axis=1, inplace=True)
df.drop(['Item_Outlet_Sales'], axis=1, inplace=True)

df = df.astype(np.float32)

scaler_X = StandardScaler(with_mean=True, with_std=True)
trX = scaler_X.fit_transform(df)

trX, tsX, trY, tsY = train_test_split(trX, trY, test_size=0.33, random_state=42)

In [10]:
total_len = trX.shape[0]

# Parameters
training_epochs = 1250
batch_size = 20
display_step = 100

# Network Parameters
n_input = trX.shape[1]
n_hidden = 150
n_output = 1

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None])

In [11]:
# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)

#     # Hidden layer with RELU activation
#     layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
#     layer_2 = tf.nn.relu(layer_2)

    # Output layer with linear activation
    out_layer = tf.add(tf.matmul(layer_1, weights['out']), biases['out'])
    return out_layer

weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden], 0, 0.05)),
    'out': tf.Variable(tf.random_normal([n_hidden, n_output], 0, 0.05))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden], 0, 0.05)),
    'out': tf.Variable(tf.random_normal([n_output], 0, 0.05))
}

In [12]:
trX = trX.astype(np.float32)
# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.sqrt(tf.reduce_mean(tf.square(pred-y)))
optimizer = tf.train.AdamOptimizer().minimize(cost)

In [13]:
# Launch the graph
sess = tf.InteractiveSession()

sess.run(tf.global_variables_initializer())

# Training cycle
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(total_len/batch_size)
    # Loop over all batches
    for i in range(total_batch-1):
        batch_x = trX[i*batch_size:(i+1)*batch_size]
        batch_y = trY[i*batch_size:(i+1)*batch_size]
        # Run optimization op (backprop) and cost op (to get loss value)
        _, c, p = sess.run([optimizer, cost, pred], feed_dict={x: batch_x,
                                                      y: batch_y})
        # Compute average loss
        avg_cost += c / total_batch

    # Display logs per epoch step
    if epoch % display_step == 0:
        print ("Epoch:", '%04d' % (epoch+1), "cost =", \
            "{:.9f}".format(avg_cost))

print ("Optimization Finished!")

Epoch: 0001 cost = 2745.095635879
Epoch: 0101 cost = 1680.443437757
Epoch: 0201 cost = 1680.030792772
Epoch: 0301 cost = 1679.882975689
Epoch: 0401 cost = 1679.807071083
Epoch: 0501 cost = 1679.761244175
Epoch: 0601 cost = 1679.726684570
Epoch: 0701 cost = 1679.698153954
Epoch: 0801 cost = 1679.673473693
Epoch: 0901 cost = 1679.647931658
Epoch: 1001 cost = 1679.627338396
Epoch: 1101 cost = 1679.608947326
Epoch: 1201 cost = 1679.591817863
Optimization Finished!


In [14]:
# Test model
error = tf.square(pred - y)
# Calculate accuracy
accuracy = tf.sqrt(tf.reduce_mean(tf.cast(error, "float")))
print(sess.run(accuracy, feed_dict={x: tsX, y: tsY}))

1676.03
